实际上，由于数据迭代器、损失函数、优化器和神经
网络层很常用，现代深度学习库也为我们实现了这些组件。

In [8]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# 读取数据集
我们可以调用框架中现有的API来读取数据。我们将features和labels作为API的参数传递，并通过数据迭代
器指定batch_size。此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内**打乱数据。**

In [9]:
def load_array(data_arrays,batch_size,is_train=True):
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

In [10]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

这里我们使用iter构造Python迭代器，并使用next从迭代器中
获取第一项。

In [11]:
next(iter(data_iter))

[tensor([[ 0.6975, -1.0601],
         [-0.2406, -0.5462],
         [-0.3236, -0.1990],
         [-0.8702,  0.0569],
         [ 0.6442,  0.9101],
         [ 0.3408, -1.3537],
         [ 0.1726, -0.4811],
         [ 0.4161, -0.5795],
         [ 0.3426, -1.4198],
         [-0.5584,  0.3639]]),
 tensor([[9.1922],
         [5.5849],
         [4.2310],
         [2.2601],
         [2.3902],
         [9.4973],
         [6.2089],
         [7.0088],
         [9.7071],
         [1.8277]])]

# 定义模型

对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。

我们首先定义一个模型变量net，它是一个Sequential类的实例。Sequential类 **将多个层串联在一起。** 当给定输入数据时，Sequential实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，以此类推。

(在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。)这一单层被称为全连接层（fully‐connected layer），因为它的每一个输入都通过矩阵‐向量乘法得到它的每个输出。

在PyTorch中，***全连接层在Linear类中定义。***

我们将两个参数传递到nn.Linear中。第一个指定**输入特征形状**，即2，第二个指定**输出**特征形状，输出特征形状为单个标量，因此为1。

In [12]:
# nn是神经网络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

# 初始化参数
我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采
样，偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样，现在我们能直接访问参数以设定它们的初始值。我
们通过net[0]选择网络中的第一个图层，然后使用weight.data和bias.data方法访问参数。我们还可以使用
替换方法normal_和fill_来重写参数值。

In [13]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

# 定义损失函数和优化算法

In [14]:
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 训练
回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data），不停地从中获取一个小批量的输
入和相应的标签。对于每一个小批量，我们会进行以下步骤:

• 通过调用net(X)生成预测并计算损失l（前向传播）。

• 通过进行反向传播来计算梯度。

• 通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [15]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000301
epoch 2, loss 0.000098
epoch 3, loss 0.000099


In [16]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([6.1977e-04, 9.4891e-05])
b的估计误差： tensor([0.0011])
